In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import gc

from datetime import datetime
import requests
import json
from pandas import json_normalize

from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])
df_ho=df_ho.reset_index()
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [3]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [4]:
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/train.csv', 'train')
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/test.csv', 'test')

train Done.
test Done.


In [5]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [6]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [204]:
t_train = train.copy()
t_test = test.copy()

In [205]:
y = t_train['target'] 

x = t_train.drop(['id','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

t_test = t_test.drop(['id','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

print(x.shape)
print(y.shape)
print(t_test.shape)

(4701217, 12)
(4701217,)
(291241, 11)


In [206]:
for i in df_ho['locdate']:
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in df_ho['locdate']:
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    
x = x.replace('월', 1)
x = x.replace('화', 2)
x = x.replace('수', 3)
x = x.replace('목', 4)
x = x.replace('금', 5)
x = x.replace('토', 6)
x = x.replace('일', 7)
t_test = t_test.replace('월', 1)
t_test = t_test.replace('화', 2)
t_test = t_test.replace('수', 3)
t_test = t_test.replace('목', 4)
t_test = t_test.replace('금', 5)
t_test = t_test.replace('토', 6)
t_test = t_test.replace('일', 7)

In [207]:
x = x.drop(['base_date','start_turn_restricted','end_turn_restricted'], axis=1)

In [208]:
x = x.drop(['start_latitude','start_longitude','end_latitude','end_longitude'], axis=1)

##### scaler = MinMaxScaler()
x[:]=scaler.fit_transform(x[:])

In [209]:
x = x.to_numpy()
y = y.to_numpy()

In [210]:
train_input, val_input, train_target, val_target = train_test_split(
    x, y, test_size=0.2, random_state=42)

In [211]:
print(train_input.shape)
print(train_target.shape)

(3760973, 5)
(3760973,)


In [212]:
train_input

array([[ 3.,  5.,  1., 60., 65.],
       [ 7., 13.,  1., 60., 40.],
       [ 2., 14.,  2., 60., 24.],
       ...,
       [ 4.,  5.,  2., 60., 45.],
       [ 1., 14.,  3., 70., 24.],
       [ 3., 13.,  2., 70., 36.]])

In [213]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [214]:
model = keras.Sequential()

In [233]:
model.add(keras.layers.Embedding(10, 5,input_length=5))
model.add(keras.layers.LSTM(8))
model.add(keras.layers.Dense(1, activation='sigmoid'))

##### model = keras.Sequential()
model.add(keras.layers.LSTM(units=20, activation='relu', return_sequences=True, input_shape=(10, 4)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.LSTM(units=20, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

In [234]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 5, 16)             8000      
                                                                 
 lstm_19 (LSTM)              (None, 8)                 800       
                                                                 
 dense_15 (Dense)            (None, 1)                 9         
                                                                 
 embedding_9 (Embedding)     (None, 1, 4)              200       
                                                                 
 lstm_20 (LSTM)              (None, 8)                 416       
                                                                 
 dense_16 (Dense)            (None, 1)                 9         
                                                                 
 embedding_10 (Embedding)    (None, 1, 4)            

##### from tensorflow.keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_input, maxlen=100)
val_seq = pad_sequences(val_input, maxlen=100)

In [235]:
train_seq = pad_sequences(train_input, maxlen=5)
val_seq = pad_sequences(val_input, maxlen=5)

In [236]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-lstm-model.h5',save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100, batch_size=456,
                    validation_data=(val_seq, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
8248/8248 [==============================] - 115s 13ms/step - loss: -144.0204 - accuracy: 8.7743e-06 - val_loss: -334.8010 - val_accuracy: 1.3826e-05
Epoch 2/100
8248/8248 [==============================] - 107s 13ms/step - loss: -465.4073 - accuracy: 8.7743e-06 - val_loss: -589.1001 - val_accuracy: 1.3826e-05
Epoch 3/100
8248/8248 [==============================] - 108s 13ms/step - loss: -711.1702 - accuracy: 8.7743e-06 - val_loss: -833.5616 - val_accuracy: 1.3826e-05
Epoch 4/100
2899/8248 [=========>....................] - ETA: 59s - loss: -876.5003 - accuracy: 8.3211e-06

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model2 = keras.Sequential()

model2.add(keras.layers.Embedding(500, 16, input_length=100))
model2.add(keras.layers.LSTM(8, dropout=0.3))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model2.compile(optimizer=rmsprop, loss='binary_crossentropy', 
               metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-dropout-model.h5', 
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                  restore_best_weights=True)

history = model2.fit(train_seq, train_target, epochs=100, batch_size=64,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model3 = keras.Sequential()

model3.add(keras.layers.Embedding(500, 16, input_length=100))
model3.add(keras.layers.LSTM(8, dropout=0.3, return_sequences=True))
model3.add(keras.layers.LSTM(8, dropout=0.3))
model3.add(keras.layers.Dense(1, activation='sigmoid'))

model3.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model3.compile(optimizer=rmsprop, loss='binary_crossentropy', 
               metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-2rnn-model.h5', 
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                  restore_best_weights=True)

history = model3.fit(train_seq, train_target, epochs=100, batch_size=64,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model4 = keras.Sequential()

model4.add(keras.layers.Embedding(500, 16, input_length=100))
model4.add(keras.layers.GRU(8))
model4.add(keras.layers.Dense(1, activation='sigmoid'))

model4.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model4.compile(optimizer=rmsprop, loss='binary_crossentropy', 
               metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-gru-model.h5', 
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                  restore_best_weights=True)

history = model4.fit(train_seq, train_target, epochs=100, batch_size=64,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
test_seq = pad_sequences(test_input, maxlen=100)

rnn_model = keras.models.load_model('best-2rnn-model.h5')

rnn_model.evaluate(test_seq, test_target)